In [1]:
import os 
os.chdir('../')
%pwd

'/Users/yashasuttangi/Developer/MachineLearning/Projects/CancerClassification/Cancer-classification-end-to-end-using-MLFlow-and-DVC'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int 
    params_batch_size: int 
    params_is_augmentation: bool 
    params_image_size: list 

In [9]:
from ChestCancerClassifier.constants import *
from ChestCancerClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf 

In [10]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model 
        params = self.params 
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "ChestCancerClassificationDataset")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [11]:
import os 
import urllib.request as request 
from zipfile import ZipFile 
import tensorflow as tf
import time 

In [12]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config 

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale= 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
        
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size 

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [13]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config = (training_config))
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e 

Reading yaml file : config/config.yaml
[2026-01-28 00:26:47,031: INFO: common]: YAML file: config/config.yaml loaded successfully
Reading yaml file : params.yaml
[2026-01-28 00:26:47,032: INFO: common]: YAML file: params.yaml loaded successfully
[2026-01-28 00:26:47,033: INFO: common]: created directory at : artifacts
[2026-01-28 00:26:47,033: INFO: common]: created directory at : artifacts/training
Found 199 images belonging to 3 classes.
Found 801 images belonging to 3 classes.
Epoch 1/10
50/50 [==============================] - 41s 813ms/step - loss: 15.7104 - accuracy: 0.4994 - val_loss: 8.9700 - val_accuracy: 0.6354
Epoch 2/10
50/50 [==============================] - 43s 870ms/step - loss: 12.7443 - accuracy: 0.5275 - val_loss: 13.3187 - val_accuracy: 0.3281
Epoch 3/10
50/50 [==============================] - 44s 880ms/step - loss: 11.3460 - accuracy: 0.5618 - val_loss: 12.4288 - val_accuracy: 0.6354
Epoch 4/10
50/50 [==============================] - 42s 847ms/step - loss: 10.033